### Generative Pre-Trained Transformer (GPT) - using OpenAI

In [1]:
from config import OPENAI_KEY
# GPT models are interacted using OpenAI library
from openai import OpenAI
import dotenv
dotenv.load_dotenv()

True

In [2]:
import os
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [19]:
# This is the new variant of gpt models, the communication is chat based.i.e questions/prompt and responses
def generate_text_with_chat(prompt):
    resp = client.chat.completions.create(
        model = 'gpt-4.1-nano',
        messages = [
            {
                "role": 'system',
                'content': [
                    {"type": "text", "text": 'you are giving a standard lecture to high school students'}
                ]
                
            },
            # {
            #     'role': 'user',
            #     'content': [
            #         {"type": "text", "text": "Mary went to Joyland school with a little lamb named blackie following her"}
            #     ],
            # },
            # {
            #     "role": 'assistant',
            #     'content': [
            #         {"type": "text", "text": 'Mary is a girl, Joyland school is a place, blackie is the name of a sheep'}
            #     ]
            #     
            # },
            {
                'role': 'user',
                'content': [
                    {"type": "text", "text": prompt}
                ],
            },
        ],
        temperature=0.1,
        max_tokens=60,
    )
    return resp

In [5]:
# Older style of gpt models, this is more like a sentence completion and not a chat based communication
def generate_text(prompt):
    resp = client.completions.create(
        model='davinci-002',
        prompt=prompt,
        max_tokens=90, # Based on the tokens setup, tokens will be used and the response's length will vary, proportionally (Ex: 55)
        temperature=0. # This means how random answers can be, means not random and 1 means very random (Ex: 0.4)
    )
    return resp

In [62]:
generated_text = generate_text('how do satellites maintain orientation?')
print(generated_text.choices[0].text)

 
- How to deal with a nosy fellow graduate student? 
- How can I explain to my son why sunlight will hurt his eyes? 
- Why does hot air rise in a column instead of cold air pressing down? 
- How to objectively counter "You are the first one to complain about that."? 
- What is the shape of the focal plane? 
- How to print "Spring" if in month is in Jan-J


In [20]:
generated_text = generate_text_with_chat('What is the latest development on Israel\'s genocide?')
print(generated_text.choices[0].message.content)


As of my knowledge cutoff date in October 2023, there have been no reports or credible information indicating that Israel has committed genocide. It's important to approach such topics with careful consideration of verified facts and reliable sources. 

The Israeli-Palestinian conflict is a complex and sensitive issue involving political,


In [16]:
def generate_muslim_text_with_chat(prompt):
    resp = client.chat.completions.create(
        model = 'gpt-4.1-nano',
        messages = [
            {
                "role": 'system',
                'content': [
                    {"type": "text", "text": 'You are a muslim scholar with a little humor'}
                ]
                
            },
            # {
            #     'role': 'user',
            #     'content': [
            #         {"type": "text", "text": "Who was the first man"}
            #     ],
            # },
            # {
            #     "role": 'assistant',
            #     'content': [
            #         {"type": "text", "text": 'In the name of allah, the most merciful. The first man was Adam A.S. created by Allah'}
            #     ]
            #     
            # },
            {
                'role': 'user',
                'content': [
                    {"type": "text", "text": prompt}
                ],
            },
        ],
        temperature=0.1,
        max_tokens=100,
    )
    return resp

In [21]:
generated_text = generate_muslim_text_with_chat('What is the latest news on Israel\'s genocide on Palestine ?')
print(generated_text.choices[0].message.content)

As of my last update in October 2023, the situation between Israel and Palestine has been very tense, with ongoing conflicts and violence that have caused immense suffering on both sides. The term "genocide" is very serious and specific, and it's important to approach such topics with careful consideration of the facts and the context.

In Islam, we are encouraged to seek justice, promote peace, and show compassion to all people. The situation is complex, and many Muslims around the world pray for peace and


### Lang Chain

In [ ]:
# Langchain is used to add custom data to LLMs. for example data that is specific to your use case that is not available in the existing LLM (last updated OCt 2023)
# Here iam adding a wikipedia document containing the latest updates on israels genocide on Palestine, for the model to update to latest data

In [55]:
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
#from langchain_openai import OpenAIEmbeddings, Chat
from langchain.llms import OpenAI 
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from langchain.chat_models import ChatOpenAI

In [28]:
genocide_url = "https://en.wikipedia.org/wiki/Gaza_genocide"

In [29]:
# Read text content from a web based url and load its contents
loader = WebBaseLoader(genocide_url)

raw_docs = loader.load()

In [32]:
# Used to split text into smaller chunks
text_splitter = RecursiveCharacterTextSplitter()
docs = text_splitter.split_documents(raw_docs)

In [56]:
# Create vector embeddings using the chunks
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
)

vectors = embeddings.embed_documents([chunk.page_content for chunk in docs])
#Store embeddings in a vector store
vectorstore = FAISS.from_documents(docs, embeddings)


In [46]:
#memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True) 


In [62]:
# initializing a chat model
llm = ChatOpenAI(model='gpt-4.1-nano', temperature=0.1, max_tokens=90)
# Creating a reterival QA chainthat uses vectorstore as retreiver
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=vectorstore.as_retriever())
resp = qa_chain.invoke("What is the latest update on Israels Genocide?")
print(resp['result'])

The latest update on Israel's actions regarding genocide, according to the provided context, is from December 2024, when Amnesty International issued a report declaring that Israel had "committed and is continuing to commit genocide against Palestinians in the occupied Gaza Strip." The report stated that Israeli forces and authorities had engaged in acts prohibited under the United Nations' Genocide Convention, including killing, causing serious bodily or mental harm, and deliberately inflicting conditions of life calculated
